# Decision Support System for Irrigation

In [1]:
# Libraries

import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import openmeteo_requests
import requests_cache
import pickle

from fuzzy_expert.variable import FuzzyVariable
from fuzzy_expert.rule import FuzzyRule
from fuzzy_expert.inference import DecompositionalInference
from retry_requests import retry
from datetime import date, timedelta
from scipy.stats import linregress
from ipywidgets import interact, widgets

import warnings

warnings.filterwarnings("ignore")

In [2]:
def pre_processing(data):

    # Columns Selection and Formatting
    
    df_rovere = data[['reading_id', 'timestamp', 'sensor_id', 'value', 'description', 'group_id']].copy()
    df_rovere[['reading_id', 'sensor_id', 'description', 'group_id']] = df_rovere[['reading_id', 'sensor_id', 'description', 'group_id']].astype(str)
    df_rovere['timestamp'] = pd.to_datetime(df_rovere['timestamp']).dt.floor('D').dt.date
    df_rovere['value'] = df_rovere['value'].astype(float)

    tens_30 = ['72', '76', '73', '74', '61', '63', '67', '65']
    tens_60 = ['71', '69', '75', '70', '62', '64', '68', '66']
    tens_all = tens_30 + tens_60
    
    df_rovere.loc[df_rovere['description'] == 'tensiometer', 'description'] = 'Tensiometer'
    df_rovere.loc[df_rovere['description'] == 'irrigation', 'description'] = 'Irrigation'

    
    # Duplication
    
    condition_not_in_list = ~df_rovere['sensor_id'].isin(tens_30)
    df_dup = df_rovere[condition_not_in_list]
    df_dup['group_id'] = df_dup['group_id'] + '_dup'

    df_rovere = df_rovere[~df_rovere['sensor_id'].isin(tens_60)]
    df_rovere = pd.concat([df_rovere, df_dup], ignore_index=True)
    df_rovere.sort_values(by=['group_id', 'timestamp'], inplace=True)
    df_rovere.reset_index(drop=True, inplace=True)

    
    # Grouping and Creation of Summary Values
    
    df_group = df_rovere.groupby(['timestamp', 'description', 'sensor_id', 'group_id']).agg({'value': ['min', 'max', 'mean', 'median', 'sum']}).reset_index()
    df_group.columns = ['timestamp', 'description', 'sensor_id', 'group_id', 'val_min', 'val_max', 'val_avg', 'val_med', 'val_sum']

    
    # Pivoting
    
    df_pivot = df_group.pivot(index=['timestamp', 'group_id'], columns='description', values=['val_min', 'val_max', 'val_avg', 'val_med', 'val_sum']).reset_index()
    df_pivot.columns = ['date', 'group_id'] + [f"{agg}_{feature}" for agg in ['min', 'max', 'avg', 'med', 'sum'] for feature in ['hum', 'temp', 'solar', 'wind', 'irr', 'rain', 'tens']]

    df = df_pivot.reset_index(drop=True)

    
    # Sensor ID Mapping
    
    group_id_mapping = {str(i): str(j) for i, j in zip(range(1, 9), tens_30)}
    group_id_mapping.update({str(i) + '_dup': str(j) for i, j in zip(range(1, 9), tens_60)})
    df['group_id'] = df['group_id'].replace(group_id_mapping)
    df = df.rename(columns={'group_id': 'sensor_id'})

    df = df[['sensor_id', 'date', 'avg_tens', 'max_temp', 'avg_hum', 'avg_solar', 'sum_rain', 'sum_irr']]
    df = df[['sensor_id', 'date', 'avg_tens'] + [col for col in df.columns if col not in ['sensor_id', 'date', 'avg_tens']]]
    df = df.sort_values(by=['sensor_id', 'date']).reset_index(drop=True)

    
    # Imputation of Missing Values
    
    float_columns = df.select_dtypes(include=['float']).columns
    df[float_columns] = df[float_columns].interpolate(method='linear', limit_direction='both')
    

    # Shifting Values using the previous 3 Days
    
    ids = df['sensor_id']
    dates = df['date']
    X = df.drop(columns=['date', 'sensor_id'])
    X = X.shift(1).add_suffix('_lag1').join(X.shift(2).add_suffix('_lag2')).join(X.shift(3).add_suffix('_lag3'))
    
    X['date'] = dates
    dates_to_remove = [date(2023, 4, 28), date(2023, 4, 29), date(2023, 4, 30)]
    X = X[~X['date'].isin(dates_to_remove)].reset_index(drop=True)
    X = X.drop(columns='date')
    
    y = df[['sensor_id', 'date', 'avg_tens']]
    y = y[~y['date'].isin(dates_to_remove)].reset_index(drop=True)
    
    df_merged = pd.concat([y, X], axis=1)
    df = df_merged[df_merged['sensor_id'].isin(tens_30)]
    
    return df


def pre_processing_target(sensor_id, target, sensors_mean):
    
    df_rovere = target.copy()
    sensor_id = str(sensor_id)

    
    # Formatting
    
    df_rovere['timestamp'] = pd.to_datetime(df_rovere['timestamp']).dt.floor('D').dt.date    
    df_rovere['sensor_id'] = df_rovere['sensor_id'].astype(str)
    df_rovere['value'] = df_rovere['value'].astype(float)
    df_rovere['description'] = df_rovere['description'].astype(str)

    df_rovere = df_rovere.sort_values(by=['timestamp', 'description']).reset_index(drop=True)

    
    # Grouping and Creation of Summary Values
    
    df_group = df_rovere.groupby(['timestamp', 'description', 'sensor_id']).agg({'value': ['min', 'max', 'mean', 'median', 'sum']}).reset_index()
    df_group.columns = ['timestamp', 'description', 'sensor_id', 'val_min', 'val_max', 'val_avg', 'val_med', 'val_sum']

    
    # Pivoting
    
    df_pivot = df_group.pivot(index=['timestamp'], columns='description', values=['val_min', 'val_max', 'val_avg', 'val_med', 'val_sum']).reset_index()
    df_pivot.columns.name = None

    if df_pivot.shape[1] == 36:
        df_pivot.columns = ['date'] + [f"{agg}_{feature}" for agg in ['min', 'max', 'avg', 'med', 'sum'] for feature in ['hum', 'temp', 'solar', 'wind', 'irr', 'rain', 'tens']]
        
    elif df_pivot.shape[1] == 31:
        df_pivot.columns = ['date'] + [f"{agg}_{feature}" for agg in ['min', 'max', 'avg', 'med', 'sum'] for feature in ['hum', 'temp', 'solar', 'irr', 'rain', 'tens']]
        
    else:
        print('Error: The Number of Features is not Correct')

    columns_to_drop = ['min_irr', 'max_irr', 'avg_irr', 'med_irr', 'min_rain', 'avg_rain', 'sum_hum', 'sum_temp', 'sum_solar', 'min_wind', 'max_wind', 'avg_wind', 'sum_wind', 'med_wind']
    columns_present = [col for col in columns_to_drop if col in df_pivot.columns]
    
    if columns_present:
        df = df_pivot.drop(columns=columns_present).reset_index(drop=True)
        
    else:
        df = df_pivot.copy()

    df = df[['date', 'avg_tens'] + [col for col in df.columns if col not in ['sensor_id', 'date', 'avg_tens']]]

    
    # Creates a Row for the next day's observation
    
    y = df['avg_tens']
    #y = y[-3:].reset_index(drop=True)
    last_date = df['date'].iloc[-1]
    target_date = last_date + timedelta(days=1)
    new_obs = [target_date] + [np.nan] * (len(df.columns) - 1)
    df.loc[len(df)] = new_obs

    X = df.drop(columns=['date'])
    X = X.shift(1).add_suffix('_lag1').join(X.shift(2).add_suffix('_lag2')).join(X.shift(3).add_suffix('_lag3'))
    X_target = pd.DataFrame(X.iloc[-1]).transpose().reset_index(drop=True)

    
    # Computation of Residuals
    
    y_mean = sensors_mean.loc[sensors_mean['sensor_id'] == sensor_id, 'sensor_mean'].values[0]
    predict_lm = [y_mean] * 3

    # Using more than 3 days
    #selected_features = stepwise_bidirectional_selection(X_train, y_train, method='aic')
    #model = sm.OLS(y_train, sm.add_constant(X_train[selected_features])).fit()
    #predict_lm =  model.predict(sm.add_constant(X[selected_features]))

    residuals = y - predict_lm
    residuals = np.append(residuals, np.nan)
    df_residuals = pd.DataFrame(residuals, columns=['residuals'])
    df_residuals = df_residuals.shift(1).add_suffix('_lag1').join(df_residuals.shift(2).add_suffix('_lag2')).join(df_residuals.shift(3).add_suffix('_lag3'))
    last_row_res = df_residuals.iloc[-1].to_frame().transpose().reset_index(drop=True)

    X_target = pd.concat([X_target, last_row_res], axis=1).sort_index(axis=1)

    return X_target


def plot_temp_analysis(df, target_feature, feature_name):

    df_target = df[target_feature]
    
    plt.figure(figsize=(20, 6))

    
    # Plot 1: Histogram of the Feature of Interest
    
    plt.subplot(1, 3, 1)
    plt.hist(df_target, bins=20, color='skyblue', edgecolor='black')
    plt.title('Distribution of ' + feature_name)
    plt.xlabel(feature_name)
    plt.ylabel('Density')
    plt.grid(True)

    
    # Plot 2: Time Series of the Feature of Interest
    
    plt.subplot(1, 3, 2)
    df_sensor_61 = df[df['sensor_id'] == '61']
    plt.plot(df_sensor_61['date'], df_sensor_61[target_feature], linestyle='-', label=feature_name, color='#636EFA')
    plt.title(feature_name + ' Time Series')
    plt.ylabel('Value')
    plt.legend()
    locator = mdates.MonthLocator(bymonthday=1)
    formatter = mdates.DateFormatter('%b')
    plt.gca().xaxis.set_major_locator(locator)
    plt.gca().xaxis.set_major_formatter(formatter)
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()

    
    # Plot 3: Scatterplot between the Feature of Interest and Tensiometer with Lineare Regression line
    
    plt.subplot(1, 3, 3)
    slope, intercept, r_value, p_value, std_err = linregress(df_target, df['avg_tens'])
    line = slope * df_target + intercept
    plt.scatter(df_target, df['avg_tens'], color='green', alpha=0.5, label='Data Points')
    plt.plot(df_target, line, color='red', label='Regression Line')
    plt.title(feature_name + ' and Tensiometer Scatterplot')
    plt.xlabel(feature_name + ' of the Previous Day')
    plt.ylabel('Avg Tensiometer Value')
    plt.legend()
    plt.grid(True)

    # Plotting
    
    plt.tight_layout()
    plt.show()


def fuzzy_decision(decision_defuzzy):
    if decision_defuzzy <= 0.5:
        return 'Not Recommended'
    elif 0.5 < decision_defuzzy <= 1.5:
        return 'Half Turn'
    elif 1.5 < decision_defuzzy <= 2.5:
        return 'Single Turn'
    else:
        return 'Double Turn'


def demo(current_avg_tensiometer, predicted_avg_tensiometer, predicted_rain_amount, predicted_max_temperature):
    plt.figure(figsize=(25,15))
    model.plot(
        variables=fuzzy_variables,
        rules=fuzzy_rules,
        current_avg_tensiometer=current_avg_tensiometer,
        predicted_avg_tensiometer=predicted_avg_tensiometer,
        predicted_rain_amount=predicted_rain_amount,
        predicted_max_temperature=predicted_max_temperature,
)
    
# def compute_indicators(data):
    
#     indicators_df = pd.DataFrame(data).copy()
#     indicators_df['sum_rain_tom'] = indicators_df['sum_rain_tom'].apply(lambda x: 0 if x <= 5 else (1 if x <= 10 else 2))
#     indicators_df['max_probrain_tom'] = indicators_df['max_probrain_tom'].apply(lambda x: 0 if x <= 40 else (1 if x <= 80 else 2))
#     indicators_df['avg_temp_tom'] = indicators_df['avg_temp_tom'].apply(lambda x: 0 if x <= 17.5 else (1 if x <= 22.5 else 2))
#     indicators_df['avg_hum_tom'] = indicators_df['avg_hum_tom'].apply(lambda x: 0 if x <= 60 else (1 if x <= 80 else 2))
#     indicators_df['avg_solar_tom'] = indicators_df['avg_solar_tom'].apply(lambda x: 0 if x <= 100 else (1 if x <= 300 else 2))

#     indicators = pd.DataFrame({
#         'tensiometer': (indicators_df['avg_tens_actual'] + indicators_df['avg_tens_trend']).astype(int),
#         'actual_water': (indicators_df['sum_rain_3_days'] + indicators_df['sum_irr_3_days']).astype(int),
#         'predicted_water': indicators_df['sum_rain_tom'] + indicators_df['max_probrain_tom'],
#         'other_factors': indicators_df['avg_temp_tom'] + indicators_df['avg_hum_tom'] + indicators_df['avg_solar_tom']
#     })
    
#     return indicators

## Exploratory Data Analysis

In [3]:
df = pd.read_json('row_data_rovere.json')
df = pre_processing(df)
df

In [4]:
selected_columns = df.drop(columns=['sensor_id', 'date'])

corr_mat = selected_columns.corr()
corr_mat.sort_values(by='avg_tens', axis=0, ascending=False, inplace=True)
corr_mat.sort_values(by='avg_tens', axis=1, ascending=False, inplace=True)

plt.figure(figsize=(40, 6))
sns.heatmap(corr_mat, annot=True, cmap='coolwarm', vmin=-1, vmax=1)

In [5]:
plot_temp_analysis(df, 'max_temp_lag1', 'Temperature')

In [6]:
plot_temp_analysis(df, 'avg_hum_lag1', 'Avg Air Humidity')

In [7]:
plot_temp_analysis(df, 'avg_solar_lag1', 'Avg Solar Radiation')

In [8]:
plot_temp_analysis(df, 'sum_rain_lag1', 'Accumulated Rain')

In [9]:
plot_temp_analysis(df, 'sum_irr_lag1', 'Sum Irrigation')

## Fuzzy System

In [10]:
excel_file = 'dss_rules.xlsx'
sheet_name = 'Rules'

rules_df = pd.read_excel(excel_file, sheet_name=sheet_name)
rules_df = rules_df.iloc[:, :-1].rename(columns=lambda x: x.lower().replace(' ', '_'))

rules_df

In [11]:
sheet_name = 'Crispy Values'

crispy_df = pd.read_excel(excel_file, sheet_name=sheet_name)
crispy_input = crispy_df.iloc[:, 0].dropna().values
crispy_output = crispy_df.iloc[:, 1].dropna().values

print(crispy_input)
print(crispy_output)

In [12]:
sheet_name = 'Fuzzy Values'

fuzzy_df = pd.read_excel(excel_file, sheet_name=sheet_name)
fuzzy_df = fuzzy_df.iloc[:, 1:].rename(columns=lambda x: x.lower().replace(' ', '_'))

fuzzy_df

In [13]:
# Fuzzy Variables Definition

fuzzy_variables = {}
n_features = int(rules_df.shape[1])

for i in range(n_features):
    
    key = rules_df.columns.tolist()[i]

    min = fuzzy_df.iloc[0, i]
    max = fuzzy_df.iloc[1, i]
    n_classes = int(fuzzy_df.iloc[2, i])
    fraction_range = (max - min) / (n_classes-1)
    universe_range = (min-2*fraction_range, max+2*fraction_range)
    
    terms = {}
    
    if i != n_features-1:
        crispy = crispy_input
        
    else:
        crispy = crispy_output
    
    for j in range(n_classes):
        
        if j == 0:
            terms[crispy[j]] = [(min, 1), (min+fraction_range, 0)]
            
        elif j == n_classes-1:
            terms[crispy[j]] = [(max-fraction_range, 0), (max, 1)]
            
        else:
            terms[crispy[j]] = [(min+(j-1)*fraction_range, 0), (min+j*fraction_range, 1), (min+(j+1)*fraction_range, 0)]
    
    value = FuzzyVariable(universe_range=universe_range, terms=terms)
    fuzzy_variables[key] = value


# Plots of Fuzzy Variables

plt.figure(figsize=(20, 12))

num_plots = len(fuzzy_variables)
num_cols = 2
num_rows = math.ceil(len(fuzzy_variables) / num_cols)

for i, (key, value) in enumerate(fuzzy_variables.items(), 1):
    plt.subplot(num_rows, num_cols, i)
    value.plot()
    plt.title(key)
    plt.xlabel('Universe Range')
    plt.ylabel('Fuzzy Value')
    plt.grid(True)

plt.tight_layout()
plt.show()

In [14]:
# Fuzzy Rules Definition

fuzzy_rules = []
n_rules = rules_df.shape[0]
n_features = rules_df.shape[1]

for i in range(n_rules):

    input_variables = []
    output_variable = []
    
    for j in range(n_features):

        if j == 0:
            term = (rules_df.columns.tolist()[j], rules_df.iloc[i, j])
            input_variables.append(term)

        elif j == n_features-1:
            term = (rules_df.columns.tolist()[j], rules_df.iloc[i, j])
            output_variable.append(term)

        elif not pd.isna(rules_df.iloc[i, j]):
            term = ('AND', rules_df.columns.tolist()[j], rules_df.iloc[i, j])
            input_variables.append(term)

        else:
            continue

    rule = FuzzyRule(premise = input_variables, consequence = output_variable)
    fuzzy_rules.append(rule)


# Fuzzy Model Definition

model = DecompositionalInference(
    and_operator='min',
    or_operator='max',
    implication_operator='Rc',
    composition_operator='max-min',
    production_link='max',
    defuzzification_operator='cog',
)

## Fuzzy Inference

### Example 1: Recommendation with manually Input Data

Case 1: Medium Current Tensiometer Value (300) & Predicted (320), No Rain Predicted, Medium Max Temperature Predicted (23 degrees) -> Irrigation Not Recommended

In [15]:
decision = model(
    variables=fuzzy_variables,
    rules=fuzzy_rules,
    current_avg_tensiometer=300,
    predicted_avg_tensiometer=320,
    predicted_rain_amount=0,
    predicted_max_temperature=25,
)

decision_defuzzy = round(decision[0]['decision'], 3)
decision_fuzzy = fuzzy_decision(decision_defuzzy)

print('Numerical Value:', decision_defuzzy)
print('Fuzzy Value:', decision_fuzzy) 

Case 2: High Current Tensiometer Value (380), Medium Predicted Tensiometer Value (320), Medium Predicted Rain Amount (10mm), High Temperature (30 degrees) -> Half Turn Recommended

In [16]:
decision = model(
    variables=fuzzy_variables,
    rules=fuzzy_rules,
    current_avg_tensiometer=380,
    predicted_avg_tensiometer=320,
    predicted_rain_amount=10,
    predicted_max_temperature=30,
)

decision_defuzzy = round(decision[0]['decision'], 3)
decision_fuzzy = fuzzy_decision(decision_defuzzy)

print('Numerical Value:', decision_defuzzy)
print('Fuzzy Value:', decision_fuzzy) 

Case 3: Medium Current Tensiometer Value (320), High Predicted Tensiometer Value (380), No Rain Predicted, High Temperature (30 degrees) -> Single Turn

In [17]:
decision = model(
    variables=fuzzy_variables,
    rules=fuzzy_rules,
    current_avg_tensiometer=320,
    predicted_avg_tensiometer=380,
    predicted_rain_amount=0,
    predicted_max_temperature=30,
)

decision_defuzzy = round(decision[0]['decision'], 3)
decision_fuzzy = fuzzy_decision(decision_defuzzy)

print('Numerical Value:', decision_defuzzy)
print('Fuzzy Value:', decision_fuzzy) 

Case 4: High Current & Predicted Tensiometer Value (380), No Rain Predicted, High Temperature (30 degrees) -> Double Turn

In [18]:
decision = model(
    variables=fuzzy_variables,
    rules=fuzzy_rules,
    current_avg_tensiometer=380,
    predicted_avg_tensiometer=380,
    predicted_rain_amount=0,
    predicted_max_temperature=30,
)

decision_defuzzy = round(decision[0]['decision'], 3)
decision_fuzzy = fuzzy_decision(decision_defuzzy)

print('Numerical Value:', decision_defuzzy)
print('Fuzzy Value:', decision_fuzzy) 

In [19]:
# All Cases

interact(
    demo,
    current_avg_tensiometer=widgets.FloatSlider(min=0, max=600),
    predicted_avg_tensiometer=widgets.FloatSlider(min=0, max=600),
    predicted_rain_amount=widgets.FloatSlider(min=0, max=30),
    predicted_max_temperature=widgets.FloatSlider(min=0, max=40),
)

### Example 2: Recommendation with Tensiometer Value Forecasting (based on an ARIMAX model) & Forecasting Data from OpenMeteo's API

In [20]:
# Current & Predicted Tensiometer Value

with open('models/75-arimax_aic-model.pkl', 'rb') as f:
    model_aic_75 = pickle.load(f)

with open('models/75-arimax_aic-fs.pkl', 'rb') as f:
    fs_aic_75 = pickle.load(f)

with open('sensors_mean.pkl', 'rb') as f:
    sensors_mean = pickle.load(f)

target_example = pd.read_json('target_example.json')
X_target = pre_processing_target(75, target_example, sensors_mean)
X_target['const'] = 1.0

current_avg_tensiometer = X_target['avg_tens_lag1'].iloc[0]
predicted_avg_tensiometer = model_aic_75.predict(X_target[['const'] + fs_aic_75])[0]


# Weather Forecasting Data (Rain Amount & Max Temperature)

cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

url = "https://api.open-meteo.com/v1/forecast"
params = {
	'latitude': 46.245079,
	'longitude': 11.164434,
	'hourly': ['rain', 'temperature_2m', 'relative_humidity_2m'],
	'timezone': 'auto',
	'start_date': '2024-02-29',
	'end_date': '2024-03-03'
}
responses = openmeteo.weather_api(url, params=params)

response = responses[0]
#print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
#print(f"Elevation {response.Elevation()} m asl")
#print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
#print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

hourly = response.Hourly()
hourly_rain = hourly.Variables(0).ValuesAsNumpy()
hourly_temperature_2m = hourly.Variables(1).ValuesAsNumpy()
#hourly_relative_humidity_2m = hourly.Variables(2).ValuesAsNumpy()

df_forecast = pd.DataFrame()
df_forecast['date'] = pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)
df_forecast['date'] = df_forecast['date'].dt.date

df_forecast['predicted_rain_amount'] = hourly_rain.astype(float)
df_forecast['predicted_max_temperature'] = hourly_temperature_2m.astype(float)
#df_forecast['predicted_air_humidity'] = hourly_relative_humidity_2m.astype(float)

first_date = df_forecast['date'].iloc[0]
last_date = df_forecast['date'].iloc[-1]
rows_to_drop = df_forecast[(df_forecast['date'] == first_date) | (df_forecast['date'] == last_date)].index
df_forecast = df_forecast.drop(rows_to_drop).reset_index(drop=True)
df_forecast = df_forecast.drop(columns=['date'])

predicted_rain_amount = df_forecast['predicted_rain_amount'].sum()
predicted_max_temperature = df_forecast['predicted_max_temperature'].max()
#predicted_air_humidity = df_forecast['predicted_air_humidity'].mean()

In [21]:
indicators = {'current_avg_tensiometer': [X_target['avg_tens_lag1'].iloc[0]], 'predicted_avg_tensiometer': [predicted_avg_tensiometer],
        'predicted_rain_amount': [predicted_rain_amount], 'predicted_max_temperature': [predicted_max_temperature]}
indicators = pd.DataFrame(indicators)

indicators

In [22]:
decision = model(
    variables=fuzzy_variables,
    rules=fuzzy_rules,
    current_avg_tensiometer=indicators.at[0, 'current_avg_tensiometer'],
    predicted_avg_tensiometer=indicators.at[0, 'predicted_avg_tensiometer'],
    predicted_rain_amount=indicators.at[0, 'predicted_rain_amount'],
    predicted_max_temperature=indicators.at[0, 'predicted_max_temperature'],
)

decision_defuzzy = round(decision[0]['decision'], 3)
decision_fuzzy = fuzzy_decision(decision_defuzzy)

print('Numerical Value:', decision_defuzzy)
print('Fuzzy Value:', decision_fuzzy) 